In [3]:
pip install git+https://github.com/Desklop/Uk_Stemmer

  Cloning https://github.com/Desklop/Uk_Stemmer to /tmp/pip-req-build-82i3hcr9
  Running command git clone -q https://github.com/Desklop/Uk_Stemmer /tmp/pip-req-build-82i3hcr9
  Created wheel for uk-stemmer: filename=uk_stemmer-1.0-py3-none-any.whl size=9512 sha256=d56f706473977cbf00edbe393d87b0faccd9c7fcff0a1f8ab6cd8998d9b834fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-rh9xuhhy/wheels/40/53/2f/982431dc2aff61a4eb52a030950bbbdef0d9178114373d6c0e
Successfully built uk-stemmer


In [4]:
pip install spark-nlp==3.4.0

     |████████████████████████████████| 140 kB 5.3 MB/s 


In [5]:
pip install pyspark==3.0.2

     |████████████████████████████████| 204.8 MB 53 kB/s 
     |████████████████████████████████| 198 kB 52.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186690 sha256=d6a4d1b27429d631ccda6a1126c2465b4e2b66ec2759e21a80b46cdebb92aa5b
  Stored in directory: /root/.cache/pip/wheels/9a/39/f6/970565f38054a830e9a8593f388b36e14d75dba6c6fdafc1ec
Successfully built pyspark


In [6]:
pip install nlu pyspark==3.0.2

     |████████████████████████████████| 478 kB 5.4 MB/s 
     |████████████████████████████████| 133 kB 54.9 MB/s 
  Attempting uninstall: spark-nlp
    Found existing installation: spark-nlp 3.4.0
    Uninstalling spark-nlp-3.4.0:
      Successfully uninstalled spark-nlp-3.4.0


In [7]:
!pip install pyenchant

     |████████████████████████████████| 55 kB 2.2 MB/s 


In [8]:
!apt-get install libenchant1c2a

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 37 not upgraded.
Need to get 1,312 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libtext-iconv-perl amd64 1.7-5build6 [13.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libaspell15 amd64 0.60.7~20110707-4ubuntu0.2 [310 kB]
Get:3 http://a

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from uk_stemmer import UkStemmer

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp

In [12]:
import nlu

In [96]:
import enchant
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
pd.options.display.float_format = "{:,.2f}".format
np.set_printoptions(suppress=True)
nltk.download('stopwords')
nltk.download('punkt')
random_state = 8080

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
ls

drive/  sample_data/


In [17]:
df = pd.read_csv("/content/drive/MyDrive/full_data.csv")

In [18]:
df.head()

,crm_customer_id,created,NAME_LOWEST
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
1,50081964,2021-01-25 14:21:07.610,Петлі меблеві
2,50081964,2021-01-29 13:53:52.920,Лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,Інсектициди
4,50081964,2021-02-20 15:02:06.730,Рейлінги для кухні


In [19]:
df.columns = ["customer_id", "date", "category"]

In [20]:
df.head(2)

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
1,50081964,2021-01-25 14:21:07.610,Петлі меблеві


In [21]:
df.shape

(1412105, 3)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1412105 entries, 0 to 1412104
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   customer_id  1412105 non-null  int64 
 1   date         1412105 non-null  object
 2   category     1412105 non-null  object
dtypes: int64(1), object(2)
memory usage: 32.3+ MB


In [23]:
print(f"number of categories: {df.category.nunique()}")

number of categories: 2129


In [24]:
print(f"number of customers: {df.customer_id.nunique()}")

number of customers: 20114


# NLP feature engineering

1. make all words lowercase
2. remove stopwords
3. preform stemming

In [25]:
df.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
1,50081964,2021-01-25 14:21:07.610,Петлі меблеві
2,50081964,2021-01-29 13:53:52.920,Лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,Інсектициди
4,50081964,2021-02-20 15:02:06.730,Рейлінги для кухні


In [26]:
df.category.unique()[:20]

array(['Петлі меблеві', 'Лотки для столових приборів', 'Інсектициди',
       'Рейлінги для кухні', 'Алюмінієвий профіль', 'Стопори дверні',
       'Зубна паста', 'Куточок алюмінієвий', 'Мішки для сміття',
       'Господарчі рукавички', 'Серветки для прибирання',
       'Аератори для крана', 'Засоби для ванних кімнат', 'Швабри',
       'Засоби для кухні', 'Засоби для догляду за унітазом',
       'Клей побутовий', 'Губки господарчі', 'Пральний порошок',
       'Клей для шпалер'], dtype=object)

In [27]:
ndf = df.drop_duplicates(subset=["customer_id", "category"])

In [28]:
print(f"distinct records: {round((ndf.shape[0] * 100) / df.shape[0])}%")

distinct records: 38%


In [29]:
ndf["category"] = ndf["category"].str.lower()

In [30]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,петлі меблеві
2,50081964,2021-01-29 13:53:52.920,лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,інсектициди
4,50081964,2021-02-20 15:02:06.730,рейлінги для кухні
6,50081964,2021-02-20 15:02:06.730,алюмінієвий профіль


In [31]:
sum(ndf.category.str.isnumeric())

0

In [32]:
categories = ndf.category.unique()

In [33]:
categories

array(['петлі меблеві', 'лотки для столових приборів', 'інсектициди', ...,
       'насоси для фонтанів', 'композитний профіль', 'дитячі шезлонги'],
      dtype=object)

In [35]:
for category in categories:
    processed_category = [word for word in word_tokenize(category)
                              if (word not in stopwords.words("ukrainian")) and word.isalpha()]
    ndf.loc[ndf["category"] == category, "category"] = ' '.join(processed_category)

In [36]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,петлі меблеві
2,50081964,2021-01-29 13:53:52.920,лотки столових приборів
3,50081964,2021-05-31 12:21:22.080,інсектициди
4,50081964,2021-02-20 15:02:06.730,рейлінги кухні
6,50081964,2021-02-20 15:02:06.730,алюмінієвий профіль


In [37]:
categories

array(['петлі меблеві', 'лотки для столових приборів', 'інсектициди', ...,
       'насоси для фонтанів', 'композитний профіль', 'дитячі шезлонги'],
      dtype=object)

In [38]:
categories = pd.Series(categories)
categories

0                                          петлі меблеві
1                            лотки для столових приборів
2                                            інсектициди
3                                     рейлінги для кухні
4                                    алюмінієвий профіль
                              ...                       
2124                                  домашні кінотеатри
2125    зарядні пристрої та блоки живлення для ноутбуків
2126                                 насоси для фонтанів
2127                                 композитний профіль
2128                                     дитячі шезлонги
Length: 2129, dtype: object

In [39]:
categories.shape

(2129,)

In [40]:
lemma_categories = nlu.load('uk.lemma').predict(categories, output_level='document')

lemma download started this may take some time.
Approximate size to download 2.3 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [41]:
lemma_categories.rename({"document": "category"}, axis=1, inplace=True)

In [42]:
lemma_categories.head()

,category,lem
0,петлі меблеві,"[петля, меблеві]"
1,лотки для столових приборів,"[лотки, для, столовий, приборів]"
2,інсектициди,[інсектициди]
3,рейлінги для кухні,"[рейлінги, для, кухня]"
4,алюмінієвий профіль,"[алюмінієвий, профіль]"


In [43]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,петлі меблеві
2,50081964,2021-01-29 13:53:52.920,лотки столових приборів
3,50081964,2021-05-31 12:21:22.080,інсектициди
4,50081964,2021-02-20 15:02:06.730,рейлінги кухні
6,50081964,2021-02-20 15:02:06.730,алюмінієвий профіль


In [44]:
ndf = ndf.merge(lemma_categories, on="category")

In [45]:
ndf

,customer_id,date,category,lem
0,50081964,2021-01-25 14:21:07.610,петлі меблеві,"[петля, меблеві]"
1,50171691,2021-02-17 13:51:00.350,петлі меблеві,"[петля, меблеві]"
2,1600035308,2021-05-09 14:28:16.517,петлі меблеві,"[петля, меблеві]"
3,2000001033,2020-10-17 14:38:24.470,петлі меблеві,"[петля, меблеві]"
4,2000001042,2021-04-20 19:50:38.070,петлі меблеві,"[петля, меблеві]"
...,...,...,...,...
337538,33000053565,2020-10-31 10:45:18.427,спортивні лави,"[спортивний, лава]"
337539,33000056666,2021-09-15 09:38:11.600,ковані хвіртки,"[кований, хвіртка]"
337540,33000074608,2021-05-09 13:18:15.007,домашні кінотеатри,"[домашній, кінотеатр]"
337541,45002082643,2021-06-29 17:45:13.720,композитний профіль,"[композитний, профіль]"


In [46]:
customers_queries = ndf.set_index(["customer_id", "date"]).drop("category", axis=1)

In [47]:
customers_queries.head()["lem"].tolist()

[['петля', 'меблеві'],
 ['петля', 'меблеві'],
 ['петля', 'меблеві'],
 ['петля', 'меблеві'],
 ['петля', 'меблеві']]

In [48]:
customers_vs_queries = pd.DataFrame(customers_queries["lem"].tolist(), index=ndf.customer_id).stack().reset_index().pivot_table(index="customer_id", columns=0, fill_value=0, aggfunc='size')

In [49]:
customers_vs_queries.head()

,bis,iгрові,lego,аварійний,авто,автоакустика,автодиагностики,автозапчастини,автокрісла,автомагнітоли,автомат,автоматика,автоматичний,автомобіль,автомобільний,автопилососи,автосигналізації,автотермометри,автотехнікс,автофарба,автохолодильники,автошампуні,автошини,агроволокно,адміністративний,аератори,аерозольні,акварельні,аквашузи,акрилові,аксесуари,аксесуарів,активний,акумулятори,акумуляторні,алмазна,алмазного,алмазні,алюмінієвий,анкери,...,шпаклівка,шпалери,шпателі,шпатлівка,штангенциркулі,штани,штемпельна,штора,штроборізи,штукатурка,штукатурні,штучний,шумовіброізоляція,шумівки,шурупи,шуруповерти,щебінь,щит,щоденник,щітки,ягода,яйцерізки,ємність,іграшка,ігровий,ізоляційні,ізоляція,інвентар,інвертори,індексні,індивідуальний,індикатор,іноземний,інсектициди,інсталяції,інструмент,інтерактивні,інформаційний,інфрачервоні,ґрунтовка
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50081964,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
50095759,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50097247,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
50113709,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50116694,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
customers_vs_queries[customers_vs_queries > 0] = 1

In [52]:
customers_vs_queries.describe()

,bis,iгрові,lego,аварійний,авто,автоакустика,автодиагностики,автозапчастини,автокрісла,автомагнітоли,автомат,автоматика,автоматичний,автомобіль,автомобільний,автопилососи,автосигналізації,автотермометри,автотехнікс,автофарба,автохолодильники,автошампуні,автошини,агроволокно,адміністративний,аератори,аерозольні,акварельні,аквашузи,акрилові,аксесуари,аксесуарів,активний,акумулятори,акумуляторні,алмазна,алмазного,алмазні,алюмінієвий,анкери,...,шпаклівка,шпалери,шпателі,шпатлівка,штангенциркулі,штани,штемпельна,штора,штроборізи,штукатурка,штукатурні,штучний,шумовіброізоляція,шумівки,шурупи,шуруповерти,щебінь,щит,щоденник,щітки,ягода,яйцерізки,ємність,іграшка,ігровий,ізоляційні,ізоляція,інвентар,інвертори,індексні,індивідуальний,індикатор,іноземний,інсектициди,інсталяції,інструмент,інтерактивні,інформаційний,інфрачервоні,ґрунтовка
count,"20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00",...,"20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00","20,066.00"
mean,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.02,0.22,0.00,0.00,0.00,0.00,0.03,0.00,0.03,0.01,0.03,0.03,0.00,0.08,0.00,0.00,0.01,0.02,0.00,0.02,0.02,0.01,0.00,0.00,0.03,0.02,0.05,...,0.08,0.12,0.11,0.01,0.00,0.00,0.00,0.08,0.00,0.05,0.03,0.03,0.02,0.01,0.14,0.01,0.00,0.00,0.03,0.05,0.00,0.00,0.06,0.02,0.08,0.09,0.02,0.01,0.00,0.01,0.00,0.00,0.00,0.03,0.00,0.03,0.01,0.00,0.00,0.11
std,0.04,0.01,0.12,0.11,0.06,0.02,0.01,0.01,0.06,0.01,0.04,0.03,0.18,0.15,0.42,0.04,0.01,0.02,0.01,0.17,0.02,0.16,0.12,0.17,0.17,0.03,0.27,0.07,0.02,0.08,0.16,0.01,0.12,0.15,0.07,0.03,0.01,0.18,0.15,0.21,...,0.27,0.32,0.32,0.09,0.06,0.04,0.04,0.28,0.01,0.21,0.17,0.17,0.15,0.09,0.34,0.12,0.06,0.06,0.17,0.23,0.06,0.06,0.24,0.13,0.27,0.29,0.15,0.08,0.05,0.09,0.02,0.02,0.01,0.16,0.05,0.17,0.09,0.01,0.06,0.32
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
75%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.

In [53]:
customers_vs_queries.to_csv("/content/drive/MyDrive/customers_vs_queries.csv")

In [85]:
docs = list(pd.Series(ndf.lem.astype(str).unique()).apply(lambda x: x[1:-1].replace("'", "").split(', ')).values)

In [89]:
docs[:10]

[['петля', 'меблеві'],
 ['інсектициди'],
 ['алюмінієвий', 'профіль'],
 ['стопори', 'дверні'],
 ['зубний', 'паста'],
 ['куточок', 'алюмінієвий'],
 ['господарчий', 'рукавичка'],
 ['швабри'],
 ['клей', 'побутовий'],
 ['губка', 'господарчий']]

In [90]:
dictionary = Dictionary(docs)

In [92]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [94]:
corpus[:10]

[[(0, 1), (1, 1)],
 [(2, 1)],
 [(3, 1), (4, 1)],
 [(5, 1), (6, 1)],
 [(7, 1), (8, 1)],
 [(3, 1), (9, 1)],
 [(10, 1), (11, 1)],
 [(12, 1)],
 [(13, 1), (14, 1)],
 [(10, 1), (15, 1)]]

In [97]:
tfidf = TfidfModel(corpus)

In [118]:
tfidf[corpus[4]]

[(7, 0.6122637205900295), (8, 0.7906536134422293)]

In [129]:
corpus[4]

[(7, 1), (8, 1)]

In [131]:
print(dictionary[7], dictionary[8])

зубний паста
